In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit.circuit import QuantumCircuit
from My_adapt import MyAdaptVQE
from copy import  deepcopy
from qiskit.circuit import Parameter,ParameterVector


# Define Molecule
molecule = MoleculeInfo(
    ["Li", "H"], [(0.0, 0.0, 0.0), (1.7, 0.0, 0.0)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)

problem = driver.run()
converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True)
transformer = FreezeCoreTransformer(
    freeze_core=True,  # how many electrons we have in our active space
    # how many orbitals we have in our active space
    remove_orbitals=[-3, -2]
)
problem = transformer.transform(problem=problem)

In [2]:
a = MyAdaptVQE(ES_problem=problem)
# circuits,oberserables = a.circuit_measurement_first()

开始初始化:挑选第一个算符...
初始化结果:第23项被选定,此项梯度最大


In [3]:
#a.run()
a.run_slover()
#a.check_nextone(order_list=[23,20,11,8,13,18,0,4])

------------第1轮正在进行中--------------
**目前已有[23]**
initial point是:[0.]
第1轮的优化结果是：[-0.1257743209178202]
目前这是第1轮,正在准备挑选下一块算符,并验证是否收敛...
没有达到收敛标准,标准为0.001,当前值为0.07653445626503247
目前还没有达到收敛！
第1轮中梯度最大项为第20项,已被选入算符池...
------------第2轮正在进行中--------------
**目前已有[23, 20]**
initial point是:[0. 0.]
第2轮的优化结果是：[0.05655771766252066, -0.1266293030333452]
目前这是第2轮,正在准备挑选下一块算符,并验证是否收敛...
没有达到收敛标准,标准为0.001,当前值为0.07334790038403899
目前还没有达到收敛！
第2轮中梯度最大项为第11项,已被选入算符池...
------------第3轮正在进行中--------------
**目前已有[23, 20, 11]**
initial point是:[0. 0. 0.]
第3轮的优化结果是：[0.054332456476837404, 0.05423535665520812, -0.1274264565066661]
目前这是第3轮,正在准备挑选下一块算符,并验证是否收敛...
没有达到收敛标准,标准为0.001,当前值为0.04309554573317497
目前还没有达到收敛！
第3轮中梯度最大项为第8项,已被选入算符池...
------------第4轮正在进行中--------------
**目前已有[23, 20, 11, 8]**
initial point是:[0. 0. 0. 0.]
第4轮的优化结果是：[0.05883633749061937, 0.058734605922843214, -0.12662440458693494, -0.032807673621613236]
目前这是第4轮,正在准备挑选下一块算符,并验证是否收敛...
没有达到收敛标准,标准为0.001,当前值为0.03399994111058256
目前还没有达到收敛！
第4轮中梯度最大项为第13项,

In [ ]:
a.run_once_solver()

In [ ]:
a.run()

In [ ]:
n = a.n_qubit
qc = QuantumCircuit(n)
qc.append(a.init_state_hf,range(n))
qc.append(a.excitation_pool_instruction[23],range(n))

a.slover.ansatz = qc
a.slover.initial_point=[0.0]
a.slover.initial_point=np.zeros(qc.num_parameters)
raw_result = a.slover.compute_minimum_eigenvalue(a.hamiltonian)

In [ ]:
display(raw_result.optimal_circuit.decompose(reps=1).draw())
raw_result.optimal_point